In [ ]:
# =========================
# [Cell 1] 필수 패키지 설치 (버전 체크 후 필요할 때만 설치)
# =========================
import sys, subprocess, os

IN_COLAB = "google.colab" in sys.modules or "COLAB_GPU" in os.environ

def ensure_pkg(spec: str):
    """
    spec 예시:
      - 'kss==4.5.4'
      - 'pyarrow==17.0.0'
    이미 해당 버전이 설치되어 있으면 건너뛴다.
    """
    import importlib, pkg_resources
    name = spec.split("==")[0]

    try:
        if "==" in spec:
            # 버전까지 고정 체크
            pkg_resources.get_distribution(spec)
        else:
            importlib.import_module(name)
        print(f"[OK] {spec} 이미 설치됨 → 설치 생략")
        return
    except Exception:
        print(f"[INSTALL] {spec} 설치 중...")

    subprocess.run(
        [sys.executable, "-m", "pip", "install", "-q", spec],
        check=True
    )
    print(f"[DONE] {spec} 설치 완료")


# ✅ 필요한 패키지 & 버전 목록
REQ_SPECS = [
    "kss==4.5.4",
    "sentence-transformers==2.6.1",
    "transformers==4.45.2",
    "accelerate==1.0.1",
    "pandas==2.2.2",
    "pyarrow==17.0.0",
    "numpy==2.0.2",   # ⬅ 여기서만 고정시켜서 binary 오류 방지
]

# 코랩에서는 numpy/torch는 그대로 쓰는 게 안전
if not IN_COLAB:
    # 로컬 윈도우 등에서만 필요하면 numpy도 고정할 수 있음
    # REQ_SPECS.insert(0, "numpy==1.26.4")
    pass

print("[INFO] 필수 패키지 버전 점검/설치 시작")
for spec in REQ_SPECS:
    ensure_pkg(spec)

print("[OK] 패키지 준비 완료. 이제 Cell 2부터 실행하면 됩니다.")


[INFO] 필수 패키지 버전 점검/설치 시작
[INSTALL] kss==4.5.4 설치 중...
[DONE] kss==4.5.4 설치 완료
[INSTALL] sentence-transformers==2.6.1 설치 중...
[DONE] sentence-transformers==2.6.1 설치 완료
[INSTALL] transformers==4.45.2 설치 중...
[DONE] transformers==4.45.2 설치 완료
[INSTALL] accelerate==1.0.1 설치 중...
[DONE] accelerate==1.0.1 설치 완료
[OK] pandas==2.2.2 이미 설치됨 → 설치 생략
[INSTALL] pyarrow==17.0.0 설치 중...
[DONE] pyarrow==17.0.0 설치 완료
[OK] numpy==2.0.2 이미 설치됨 → 설치 생략
[OK] 패키지 준비 완료. 이제 Cell 2부터 실행하면 됩니다.


In [ ]:
# =========================
# [Cell 2] 환경 점검 (GPU/RAM)
# =========================
import sys, os, platform

IN_COLAB = "google.colab" in sys.modules or "COLAB_GPU" in os.environ

import numpy as np, pandas as pd
import transformers, accelerate, kss

print("Env:", "Colab" if IN_COLAB else platform.platform())
print("numpy       :", np.__version__)
print("pandas      :", pd.__version__)
print("transformers:", transformers.__version__)
print("accelerate  :", accelerate.__version__)
print("kss         :", kss.__version__)

os.environ["TOKENIZERS_PARALLELISM"] = "false"

# GPU 정보
try:
    import torch
    print("CUDA available:", torch.cuda.is_available())
    if torch.cuda.is_available():
        print("GPU:", torch.cuda.get_device_name(0))
except Exception as e:
    print("[INFO] torch 미사용 또는 미설치:", e)


Env: Colab
numpy       : 2.0.2
pandas      : 2.2.2
transformers: 4.45.2
accelerate  : 1.0.1
kss         : 4.5.4
CUDA available: True
GPU: NVIDIA A100-SXM4-40GB


In [ ]:
# =========================
# [Cell 3] Google Drive 마운트 & 작업 폴더
# =========================
import os, pathlib, sys

IN_COLAB = "google.colab" in sys.modules or "COLAB_GPU" in os.environ

if IN_COLAB:
    from google.colab import drive  # type: ignore
    drive.mount('/content/drive', force_remount=True)
    BASE_DIR = "/content/drive/MyDrive/colab"
else:
    BASE_DIR = os.path.expanduser("~/colab")

OUT_DIR = os.path.join(BASE_DIR, "output")
pathlib.Path(BASE_DIR).mkdir(parents=True, exist_ok=True)
pathlib.Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

print("BASE_DIR:", BASE_DIR)
print("OUT_DIR :", OUT_DIR)


Mounted at /content/drive
BASE_DIR: /content/drive/MyDrive/colab
OUT_DIR : /content/drive/MyDrive/colab/output


In [ ]:
# =========================
# [Cell 4] 파일 경로 설정 (MyDrive 전체에서 사전/사후 엑셀 찾기, 한글 정규화 포함)
# =========================
import os, glob, unicodedata

# CSV 예비 이름 (단일 설문용, 없어도 무관)
SURVEY_BASENAME = "Survey.csv"
CSV_PATH = os.path.join(BASE_DIR, SURVEY_BASENAME)

# 검색 루트: BASE_DIR의 상위 (예: /content/drive/MyDrive)
search_root = os.path.dirname(BASE_DIR)
print("search_root:", search_root)

# === MyDrive 전체에서 .xlsx 다 찾기 ===
xlsx_candidates = glob.glob(os.path.join(search_root, "**", "*.xlsx"), recursive=True)

print("=== .xlsx 전체 후보 (일부만 표시) ===")
if not xlsx_candidates:
    print(" (없음)")
else:
    for p in xlsx_candidates[:20]:
        rel = p.replace(search_root + "/", "")
        print(" -", rel)
    if len(xlsx_candidates) > 20:
        print(f" ... (총 {len(xlsx_candidates)}개)")

# ⬇ Unicode 정규화(NFC) 후, '교육연구프로그램' + '설문' + '응답' 포함 & '_analysis' 제외
xlsx_survey = []
for p in xlsx_candidates:
    fname = os.path.basename(p)
    fname_norm = unicodedata.normalize("NFC", fname)
    if ("교육연구프로그램" in fname_norm) and ("설문" in fname_norm) and ("응답" in fname_norm) \
       and ("_analysis" not in fname_norm):
        xlsx_survey.append(p)

print("\n=== 설문(응답) 원본 후보(.xlsx) ===")
if not xlsx_survey:
    print(" (없음)")
else:
    for p in xlsx_survey:
        rel = p.replace(search_root + "/", "")
        fname_norm = unicodedata.normalize("NFC", os.path.basename(p))
        print(" -", fname_norm, "←", rel)

XLSX_PRE_PATH  = None
XLSX_POST_PATH = None

# 파일명(정규화된 fname_norm)에 '사전', '사후' 들어가는지로 구분
for path in xlsx_survey:
    fname_norm = unicodedata.normalize("NFC", os.path.basename(path))
    if "사후" in fname_norm:
        XLSX_POST_PATH = path
    if "사전" in fname_norm:
        XLSX_PRE_PATH = path

# 예외: '사전' 표시가 없고 후보가 1개뿐이면 → 사전으로 가정
if XLSX_PRE_PATH is None and len(xlsx_survey) == 1:
    XLSX_PRE_PATH = xlsx_survey[0]

print("\nXLSX_PRE_PATH :", XLSX_PRE_PATH if XLSX_PRE_PATH else "None")
print("XLSX_POST_PATH:", XLSX_POST_PATH if XLSX_POST_PATH else "None")
print("CSV_PATH      :", CSV_PATH, "→", "OK" if os.path.isfile(CSV_PATH) else "NOT FOUND")

# 우선순위: 엑셀(사전/사후 둘 중 하나라도 있으면) > CSV
if (XLSX_PRE_PATH and os.path.isfile(XLSX_PRE_PATH)) or \
   (XLSX_POST_PATH and os.path.isfile(XLSX_POST_PATH)):
    SURVEY_SOURCE = "xlsx"
    SURVEY_PATH   = None   # 엑셀은 여러 개 합칠 거라 단일 경로 안 씀
elif os.path.isfile(CSV_PATH):
    SURVEY_SOURCE = "csv"
    SURVEY_PATH   = CSV_PATH
else:
    SURVEY_SOURCE = None
    SURVEY_PATH   = None

print("SURVEY_SOURCE :", SURVEY_SOURCE)

# REF 후보 파일 (BASE_DIR 바로 아래에서만 탐색)
REF_CANDIDATES = [
    "REF.txt", "학습용ref.txt", "Reference.txt",
    "Ref.txt", "REF.md", "reference.md"
]
REF_PATH = next(
    (os.path.join(BASE_DIR, n) for n in REF_CANDIDATES
     if os.path.isfile(os.path.join(BASE_DIR, n))),
    None
)
print("REF_PATH      :", REF_PATH if REF_PATH else "(미발견: 같은 폴더에 REF.txt/학습용ref.txt 두면 자동 인식)")


search_root: /content/drive/MyDrive
=== .xlsx 전체 후보 (일부만 표시) ===
 - colab/2025 교육연구프로그램 사후 설문(응답).xlsx
 - colab/2025 교육연구프로그램 사전 설문(응답).xlsx

=== 설문(응답) 원본 후보(.xlsx) ===
 - 2025 교육연구프로그램 사후 설문(응답).xlsx ← colab/2025 교육연구프로그램 사후 설문(응답).xlsx
 - 2025 교육연구프로그램 사전 설문(응답).xlsx ← colab/2025 교육연구프로그램 사전 설문(응답).xlsx

XLSX_PRE_PATH : /content/drive/MyDrive/colab/2025 교육연구프로그램 사전 설문(응답).xlsx
XLSX_POST_PATH: /content/drive/MyDrive/colab/2025 교육연구프로그램 사후 설문(응답).xlsx
CSV_PATH      : /content/drive/MyDrive/colab/Survey.csv → NOT FOUND
SURVEY_SOURCE : xlsx
REF_PATH      : /content/drive/MyDrive/colab/REF.txt


In [ ]:
# =========================
# [Cell 5] 데이터 로드 & 미리보기 (CSV/엑셀 자동)
# =========================
import pandas as pd
from IPython.display import display
import os

if SURVEY_SOURCE is None:
    raise FileNotFoundError(
        "설문 파일을 찾지 못했습니다.\n"
        f"- CSV          : {CSV_PATH}\n"
        f"- XLSX(사전 추정): {XLSX_PRE_PATH}\n"
        f"- XLSX(사후 추정): {XLSX_POST_PATH}"
    )

if SURVEY_SOURCE == "csv":
    # CSV 단일 설문
    try:
        df = pd.read_csv(SURVEY_PATH, encoding="utf-8-sig")
    except Exception:
        df = pd.read_csv(SURVEY_PATH, sep="\t", encoding="utf-8-sig")

    df["설문시점"]   = "단일"
    df["source_file"] = os.path.basename(SURVEY_PATH)
    print("[INFO] Survey loaded from CSV:", SURVEY_PATH)

else:
    # 엑셀(사전/사후) 여러 파일 통합
    frames = []
    loaded_files = []

    if XLSX_PRE_PATH and os.path.isfile(XLSX_PRE_PATH):
        df_pre = pd.read_excel(XLSX_PRE_PATH)
        df_pre["설문시점"]   = "사전"   # ★ 사전 표시
        df_pre["source_file"] = os.path.basename(XLSX_PRE_PATH)
        frames.append(df_pre)
        loaded_files.append(os.path.basename(XLSX_PRE_PATH))

    if XLSX_POST_PATH and os.path.isfile(XLSX_POST_PATH):
        df_post = pd.read_excel(XLSX_POST_PATH)
        df_post["설문시점"]   = "사후"  # ★ 사후 표시
        df_post["source_file"] = os.path.basename(XLSX_POST_PATH)
        frames.append(df_post)
        loaded_files.append(os.path.basename(XLSX_POST_PATH))

    if not frames:
        raise FileNotFoundError(
            "엑셀 설문 파일(XLSX_PRE_PATH/XLSX_POST_PATH)을 찾지 못했습니다.\n"
            f"- XLSX_PRE_PATH : {XLSX_PRE_PATH}\n"
            f"- XLSX_POST_PATH: {XLSX_POST_PATH}"
        )

    df = pd.concat(frames, ignore_index=True)
    print("[INFO] Survey loaded from Excel files:", loaded_files)

print("Survey shape:", df.shape)
print("Survey columns:", list(df.columns)[:16], "...")
display(df.head(3))

# REF 텍스트 로딩
ref_text = None
if REF_PATH and os.path.isfile(REF_PATH):
    with open(REF_PATH, "r", encoding="utf-8") as f:
        ref_text = f.read()
    print(f"Loaded REF: {os.path.basename(REF_PATH)} (chars={len(ref_text)})")
else:
    print("[알림] REF 텍스트 미탐지 → RAG 미사용")


[INFO] Survey loaded from Excel files: ['2025 교육연구프로그램 사전 설문(응답).xlsx', '2025 교육연구프로그램 사후 설문(응답).xlsx']
Survey shape: (21, 14)
Survey columns: ['타임스탬프', '이름을 적어주세요.(예시: 박민준)', '현재 몇 학년에 재학 중이신가요?', '고등학교에서 수강한 과목을 "모두" 골라주세요.', "자신이 배웠던 지구과학1, 지구과학2 교과서의 출판사를 골라주세요. (두 교과서의 출판사가 달랐다면, '기타'를 고르고 각각 적어주세요.)", '1. 암흑물질(Dark Matter)과 암흑에너지(Dark Energy)를 각각 정의하고, 우주 전체에 미치는 영향의 가장 큰 차이점을 설명하세요.', '2. 암흑물질의 존재를 강력하게 지지하는 천문학적 관측 증거를 아는 대로 서술하세요.', "3. 암흑물질과 암흑에너지를 '암흑(Dark)'이라고 부르는 이유가 무엇이라고 생각하나요?", '4. 암흑물질의 후보로 언급되는 입자들의 특징을 설명하고, 왜 그런 특징을 보이는지 설명하세요.', "5. 1990년대 후반, 천문학자들이 우주의 '가속 팽창'을 발견하게 된 결정적인 관측은 무엇이었으며, 그 관측 결과가 왜 가속 팽창을 의미하는지 설명하세요.", "6. '암흑물질과 암흑에너지는 과학자들이 자신들의 이론이 틀린 것을 감추기 위해 임의로 도입한 개념(fudge factor)에 불과하다'라는 주장에 대해 어떻게 생각하시나요? 자신의 견해를 과학적 증거에 기반하여 논하세요.", '7. 학생들에게 암흑물질과 암흑에너지를 설명하기 위한 효과적인 비유나 예시를 한 가지 제시해 주세요.', '설문시점', 'source_file'] ...


,타임스탬프,이름을 적어주세요.(예시: 박민준),현재 몇 학년에 재학 중이신가요?,"고등학교에서 수강한 과목을 ""모두"" 골라주세요.","자신이 배웠던 지구과학1, 지구과학2 교과서의 출판사를 골라주세요. (두 교과서의 출판사가 달랐다면, '기타'를 고르고 각각 적어주세요.)","1. 암흑물질(Dark Matter)과 암흑에너지(Dark Energy)를 각각 정의하고, 우주 전체에 미치는 영향의 가장 큰 차이점을 설명하세요.",2. 암흑물질의 존재를 강력하게 지지하는 천문학적 관측 증거를 아는 대로 서술하세요.,3. 암흑물질과 암흑에너지를 '암흑(Dark)'이라고 부르는 이유가 무엇이라고 생각하나요?,"4. 암흑물질의 후보로 언급되는 입자들의 특징을 설명하고, 왜 그런 특징을 보이는지 설명하세요.","5. 1990년대 후반, 천문학자들이 우주의 '가속 팽창'을 발견하게 된 결정적인 관측은 무엇이었으며, 그 관측 결과가 왜 가속 팽창을 의미하는지 설명하세요.",6. '암흑물질과 암흑에너지는 과학자들이 자신들의 이론이 틀린 것을 감추기 위해 임의로 도입한 개념(fudge factor)에 불과하다'라는 주장에 대해 어떻게 생각하시나요? 자신의 견해를 과학적 증거에 기반하여 논하세요.,7. 학생들에게 암흑물질과 암흑에너지를 설명하기 위한 효과적인 비유나 예시를 한 가지 제시해 주세요.,설문시점,source_file
0,2025-10-21 16:39:27.190,김지혜,대학교 2학년,"화학 1, 생명과학 1, 생명과학 2, 지구과학 1, 지구과학 2",비상교육,"암흑물질은 눈에 보이지 않지만, 질량을 가진 물질을 의미한다. 암흑에너지는 우주를 ...","은하의 회전속도, 미세중력탐사",눈에 보이지 않기 때문에,"윔프, 윔프는 질량이 무겁고, 다른 입자들과 상호작용이 적기 때문이다.","Ia 초신성 관측, 이 초신성은 표준광원인데, 그렇기 때문에 절대등급을 알 고 있다...","아니다. 은하의 회전속도, 은하단의 중력렌즈현상을 비추어보았을 때, 현대우주론을 발...","눈에 보이지 않지만 영향을 주는 존재,, 녹아서 없어지는 것들? 그치만 영향을 주는...",사전,2025 교육연구프로그램 사전 설문(응답)....
1,2025-10-21 17:12:34.769,이도향,대학교 1학년,"물리학 1, 물리학 2, 화학 1, 화학 2, 생명과학 1, 지구과학 1, 지구과학...",동아,암흑 물질 : 우주 팽창에서 인력으로 작용하는 물질\n암흑 에너지 : 우주 팽창에서...,은하의 회전 속도(무슨 은하인지는 모르겠어요),"존재는 확실하지만, 관측 불가능하기 때문",모름,"우주 평탄성, 우주 지평선, 자기홀극 등등\n우주 지평선 -> 우주 배경복사가 가장...","아직 밝혀지지 않은 개념을, 관측 결과를 뒷받침하려 일부로 만든 개념이기에 맞는 말...",우리 실생활 공기처럼 비유하지 않을까요…?,사전,2025 교육연구프로그램 사전 설문(응답)....
2,2025-10-21 18:16:59.803,김차윤,대학교 2학년,"물리학 1, 물리학 2, 생명과학 1, 생명과학 2, 지구과학 1, 지구과학 2, ...",비상교육,우주의 대부분을 차지.,우주가 팽창한다.,어두워서,몰라요ㅠㅠ,몰라요ㅠㅠ,몰라요ㅠㅠ,흠.. 풍선에 스터키 붙여서 바람넣기,사전,2025 교육연구프로그램 사전 설문(응답)....


Loaded REF: REF.txt (chars=4561)


In [ ]:
# =========================
# [Cell 6] 임베딩 RAG (CPU 강제) — topk_context 정의
# =========================
from sentence_transformers import SentenceTransformer
import unicodedata

def chunk_text(text: str, max_chars=800):
    if not text:
        return []
    text = unicodedata.normalize("NFKC", str(text))
    parts, buf = [], []
    for line in text.splitlines():
        nxt = ("\n".join(buf + [line])).strip()
        if len(nxt) > max_chars:
            if buf:
                parts.append("\n".join(buf).strip())
            buf = [line]
        else:
            buf.append(line)
    if buf:
        parts.append("\n".join(buf).strip())
    return [p for p in parts if len(p) >= 40]

EMBED_MODEL_ID = "intfloat/multilingual-e5-small"
embed = SentenceTransformer(EMBED_MODEL_ID, device="cpu")
print("Embedding model:", EMBED_MODEL_ID)

ref_chunks, ref_vec = [], None
if ref_text:
    ref_chunks = chunk_text(ref_text, max_chars=800)
    if ref_chunks:
        ref_vec = embed.encode(ref_chunks, normalize_embeddings=True)
        print("REF chunks:", len(ref_chunks), "| vectors:", getattr(ref_vec, "shape", None))
    else:
        print("[참고] REF는 있으나 chunk=0 → RAG 미사용")
else:
    print("[참고] REF 없음 → RAG 미사용")

def topk_context(query: str, k=4, max_chars=1200):
    if (ref_vec is None) or (not ref_chunks) or (not query):
        return ""
    qv = embed.encode([query], normalize_embeddings=True, convert_to_numpy=True)[0]
    sims = (ref_vec @ qv).tolist()
    idx = sorted(range(len(ref_chunks)), key=lambda i: sims[i], reverse=True)[:k]
    ctx = "\n\n".join(ref_chunks[i] for i in idx)
    return ctx[:max_chars]


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

Embedding model: intfloat/multilingual-e5-small
REF chunks: 7 | vectors: (7, 384)


In [ ]:
# =========================
# [Cell 7] LLM 로드 — 7B 우선, OOM 시 3B 폴백
# =========================
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

USE_7B = True
MODEL_7B = "Qwen/Qwen2.5-7B-Instruct"
MODEL_3B = "Qwen/Qwen2.5-3B-Instruct"

prefer_bf16 = torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] >= 8
dtype = torch.bfloat16 if prefer_bf16 else torch.float16

def _load(model_id: str):
    tok = AutoTokenizer.from_pretrained(model_id, use_fast=True)
    if tok.pad_token_id is None:
        tok.pad_token_id = tok.eos_token_id
    mdl = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=(dtype if torch.cuda.is_available() else torch.float32),
        device_map="auto",
        low_cpu_mem_usage=True,
    )
    return tok, mdl

try:
    target = MODEL_7B if USE_7B else MODEL_3B
    tok, mdl = _load(target)
    print("LLM loaded:", target, "| dtype:", dtype)
except RuntimeError:
    print(f"[경고] {target} 로드 실패 → 3B 폴백")
    tok, mdl = _load(MODEL_3B)
    target = MODEL_3B
    print("LLM loaded:", target)

def llm_generate(prompt: str, max_new_tokens=220):
    inputs = tok(prompt, return_tensors="pt").to(mdl.device)
    with torch.no_grad():
        out = mdl.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            temperature=0.0,
            top_p=1.0,
            eos_token_id=tok.eos_token_id,
            pad_token_id=tok.pad_token_id,
        )
    gen_ids = out[0][inputs["input_ids"].shape[1] :]
    return tok.decode(gen_ids, skip_special_tokens=True).strip()


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

LLM loaded: Qwen/Qwen2.5-7B-Instruct | dtype: torch.bfloat16


In [ ]:
# ============================================
# === Cell 8. 라벨 / 규칙 / 도우미 정의   ===
# ============================================
import re, json, unicodedata
try:
    _ = kss.split_sentences("테스트")
    _KSS_OK = True
except Exception:
    _KSS_OK = False

# -------- 라벨 & HEAVY 라벨 --------
ALLOWED_LABELS = [
    "용어-암흑 의미혼동",
    "DM-DE 혼동",
    "관측/검출 혼동",
    "분포/역할 오해",
    "기전/정의 불충분",
    "일상어/동음이의 착각",
    "비후보/천체 혼동",
    "음모/단정",
    "무응답/비관련",
    "없음(정상)"
]

HEAVY_LABELS = {
    "용어-암흑 의미혼동",
    "DM-DE 혼동",
    "비후보/천체 혼동",
    "음모/단정",
    "일상어/동음이의 착각",
}

# -------- 문항별 피드백 템플릿 --------
Q_FEEDBACK_MAP = {
    "Q1": {
        "ok": "암흑물질=비발광(EM 상호작용 미약)·구조를 모음 / 암흑에너지=우주 가속 성분(음의 압력)으로 ‘결속 vs 가속’을 한 줄로 대비하세요.",
        "by_label": {
            "DM-DE 혼동": "DM은 인력·구조 형성, DE는 팽창 가속(음의 압력)로 구분하세요.",
            "용어-암흑 의미혼동": "‘암흑’이라는 이름의 뜻을 ‘어두워서/무서워서/미지라서’가 아니라 EM과 거의 상호작용하지 않아 비발광이라는 의미로 정리하세요.",
            "분포/역할 오해": "DE는 큰 스케일에서 균일(우주상수 가정)로 모델링, DM은 비균일 분포로 구조 형성에 기여함을 분리해 적으세요."
        }
    },
    "Q2": {
        "ok": "회전곡선 평탄·중력렌즈·은하단 동역학·CMB 중 1–2개를 축/오차와 함께 제시하세요.",
        "by_label": {
            "기전/정의 불충분": "증거는 ‘관측→해석→주장’ 사슬로 서술하고, 그래프 축/오차를 한 줄로 명시하세요."
        }
    },
    "Q3": {
        "ok": "‘직접’은 어렵지만 중력 효과로 ‘간접 검출’ 가능함을 한 줄로 정리하세요.",
        "by_label": {
            "관측/검출 혼동": "‘관측/검출 불가능’이라고 단정하기보다는 ‘직접 검출은 어렵지만 중력 효과로 간접 검출 가능’으로 표현하세요.",
            "용어-암흑 의미혼동": "dark/암흑이라는 이름의 뜻을 ‘정체불명/미지’가 아니라 EM 상호작용이 거의 없어 비발광이라는 의미로 구분해서 적으세요."
        }
    },
    "Q4": {
        "ok": "WIMP/액시온 등 ‘입자’ 후보 1–2개 + ‘약한 상호작용·질량 범위’가 관측/우주론 제약에서 온다는 이유를 연결하세요.",
        "by_label": {
            "비후보/천체 혼동": "후보는 ‘입자’입니다. 블랙홀·항성·퀘이사 등 거대천체 일반화는 피하세요."
        }
    },
    "Q5": {
        "ok": "SN Ia 표준촛불: 더 어둡게 관측→더 멀다→과거 감속→지금 가속. 축/오차/모형가정을 한 줄로 언급하세요.",
        "by_label": {
            "기전/정의 불충분": "SN Ia 표준촛불 사슬(밝기–거리–적색편이–가속)을 빠짐없이 적으세요."
        }
    },
    "Q6": {
        "ok": "회전곡선·렌즈·CMB/BAO·SN Ia 등 ‘독립 관측의 합치’ + 대안(MOND 등) 한계를 1줄로 평가하세요.",
        "by_label": {
            "음모/단정": "‘속이기 위해 만든 개념’인지, 여러 관측(총알은하단·CMB 등)을 동시에 설명하는지 근거로 평가하세요."
        }
    },
    "Q7": {
        "ok": "비유 1개를 제시하고, 이 비유가 어디까지 맞고 어디서 실제 암흑물질/에너지와 달라지는지 한계 문장을 함께 쓰세요.",
        "by_label": {}
    }
}

# -------- 개념 태그 한글 이름 --------
POS_NAME_KO = {
    "DM=전자기 비상호작용(비발광)": "DM 비발광(EM 상호작용 미약)",
    "DM=중력효과 간접 검출": "DM 간접 검출(중력 효과)",
    "DE=가속팽창/음의 압력": "DE 가속팽창(음의 압력)",
    "DM↔DE 대비": "DM↔DE 대비",
    "회전곡선 평탄": "회전곡선 평탄",
    "렌즈/질량지도": "중력렌즈/질량지도",
    "은하단 동역학": "은하단 동역학",
    "CMB/구조형성": "CMB/대규모구조",
    "핵심 사슬": "SN Ia 핵심 사슬",
    "다중근거": "다중 관측 근거",
    "대안 한계": "대안이론 한계",
    "비유 제시": "비유 제시"
}

def format_concept_tags(mislabels_list, pos_hits_list):
    tags = []
    for p in (pos_hits_list or []):
        nm = POS_NAME_KO.get(p, p)
        tags.append(f"(정) {nm}")
    for m in (mislabels_list or []):
        if m == "없음(정상)":
            continue
        tags.append(f"(부) {m}")
    out, seen = [], set()
    for t in tags:
        if t not in seen:
            seen.add(t)
            out.append(t)
    return "; ".join(out[:8])

def compose_feedback_qaware(qid: str, mislabels, pos_hits):
    cfg = Q_FEEDBACK_MAP.get(qid, {})
    tips, by_label = [], cfg.get("by_label", {})
    for lb in (mislabels or []):
        if lb in by_label:
            tips.append(by_label[lb])
    ok = cfg.get("ok")
    if ok:
        tips.append(ok)
    seen, out = set(), []
    for t in tips:
        if not t:
            continue
        k = t.strip()
        if k in seen:
            continue
        seen.add(k); out.append(k)
        if len(out) >= 2:
            break
    if not out:
        return "핵심 포인트를 한 문장으로 정리해 보세요."
    if len(out) == 1:
        return out[0]
    return " ".join(out[:2])

# -------- Q별 허용 라벨 (Q7: 오개념 라벨 금지) --------
Q_ALLOWED_MISLABELS = {
    "Q1": ALLOWED_LABELS,
    "Q2": [x for x in ALLOWED_LABELS if x != "용어-암흑 의미혼동"],
    "Q3": [x for x in ALLOWED_LABELS if x != "비후보/천체 혼동"],
    "Q4": [x for x in ALLOWED_LABELS if x != "용어-암흑 의미혼동"],
    "Q5": [x for x in ALLOWED_LABELS if x not in ("용어-암흑 의미혼동", "비후보/천체 혼동")],
    "Q6": ALLOWED_LABELS,
    "Q7": ["무응답/비관련", "없음(정상)"],
}

# -------- Q_INTENT / 무응답 / 규칙 스캐너 --------
def _norm(s):
    return re.sub(r"\s+", " ", str(s or "")).strip().lower()

def has_any(s, terms):
    return any(t.lower() in s for t in terms)

POS_CORE = {
    "Q1": {"DM=전자기 비상호작용(비발광)", "DM=중력효과 간접 검출", "DE=가속팽창/음의 압력", "DM↔DE 대비"},
    "Q2": {"회전곡선 평탄", "렌즈/질량지도", "은하단 동역학", "CMB/구조형성"},
    "Q3": {"dark=EM 비상호작용", "간접검출"},
    "Q4": {"후보 명시"},
    "Q5": {"핵심 사슬"},
    "Q6": {"다중근거", "대안 한계"},
    "Q7": {"비유 제시"}
}

Q_INTENT = {
  "Q1": {
    "intent": ["정의", "차이", "구분", "구별", "비교", "대비", "반면", "vs"],
    "pos": {
      # DM 정의/역할 정개념
      "DM=전자기 비상호작용(비발광)": [
          "전자기력", "전자기파", "빛",
          "발광하지 않", "비발광",
          "전자기 상호작용 미약", "EM 상호작용",
          "전자기장"
      ],
      "DM=중력효과 간접 검출": ["중력", "중력 렌즈", "회전곡선", "질량-광도", "질량대 광도", "간접"],
      "DE=가속팽창/음의 압력": ["가속 팽창", "우주 상수", "우주상수", "ΛCDM", "람다", "음의 압력", "균일 분포", "일정한 밀도"],
      "DM↔DE 대비": ["모음", "구속", "구조 형성", "팽창 가속", "감속", "대비", "반면", "vs", "차이", "구분"]
    },
    "neg": {
      "DM-DE 혼동": [
          "암흑물질이 가속", "암흑 에너지가 구속", "암흑에너지가 구속", "DE가 인력", "DM이 척력"
      ],
      # ⚠ 여기서는 '미지/정체불명' 같은 말만으로는 오개념 처리하지 않도록 제한.
      "용어-암흑 의미혼동": [
          "어두워서 암흑", "검어서 암흑", "까매서 암흑",
          "무서워서 암흑", "공포", "공포스러운",
          "암흑이라는 이름은 어두워서", "dark라는 이름은 어두워서"
      ],
      "관측/검출 혼동": ["검출 불가능", "절대 관측 불가"]
    }
  },
  "Q2": {
    "intent": ["증거", "관측", "근거", "그래프"],
    "pos": {
      "회전곡선 평탄": ["회전속도", "회전 속도", "회전 곡선", "회전곡선", "케플러", "강체 회전", "원반 회전", "평탄"],
      "렌즈/질량지도": ["중력 렌즈", "약렌즈", "질량지도", "총알 은하단", "weak lens"],
      "은하단 동역학": ["은하단 속도", "virial", "자유낙하", "질량 결손", "mass deficit"],
      "CMB/구조형성": ["CMB", "요동", "BAO", "구조 형성", "파워 스펙트럼"]
    },
    "neg": {
      "기전/정의 불충분": ["빅뱅만", "팽창만", "가속만", "블랙홀만"]
    }
  },
  "Q3": {
    "intent": ["이유", "의미", "왜", "부르"],
    "pos": {
      "dark=EM 비상호작용": [
          "전자기파로", "빛으로",
          "발광하지 않", "비발광",
          "전자기 상호작용 미약", "전파/가시광 미검출",
          # ✅ 튜닝: '전자기파와 상호작용하지 않는다' 류의 표현을 확실히 포착
          "전자기파와 상호작용하지 않",
          "전자기파와 잘 상호작용하지 않",
          "빛과 상호작용하지 않",
          "빛과 잘 상호작용하지 않",
          "빛과 거의 상호작용하지 않",
          "전자기파와 거의 상호작용하지 않"
      ],
      "간접검출": ["중력", "렌즈", "회전곡선", "간접", "질량 분포"]
    },
    "neg": {
      # 여기서는 이름의 '뜻'을 미지/무서움으로 설명할 때만 잡는다.
      "용어-암흑 의미혼동": [
          "어두워서 암흑", "검아서 암흑", "까매서 암흑",
          "무서워서 암흑", "공포스러워서 암흑",
          "정체불명이라서 암흑", "정체를 알 수 없어서 암흑", "미지라서 암흑",
          "dark라는 이름은 정체를 알 수 없어서", "dark라는 이름은 미지라서"
      ],
      # '직접 관측 불가' 류는 주변에 중력/간접 얘기 나오면 가급적 무시
      "관측/검출 혼동": [
          "관측 불가능", "관측이 불가능",
          "검출 불가능", "절대 관측 불가",
          "이론으로만 존재", "이론상으로만 존재"
      ]
    }
  },
  "Q4": {
    "intent": ["후보", "입자", "WIMP", "윔프", "AXION", "액시온", "MACHO", "마초"],
    "pos": {
        "후보 명시": ["윔프", "wimp", "액시온", "axion", "비바리온", "macho", "중성미자"]
    },
    "neg": {
        "비후보/천체 혼동": ["행성", "항성", "블랙홀", "중성자별", "퀘이사", "은하"]
    }
  },
  "Q5": {
    "intent": ["가속팽창", "SN Ia", "초신성", "표준 촛불", "허블도표", "거리지수", "허블"],
    "pos": {
      "핵심 사슬": ["Ia", "초신성", "표준 촛불", "겉보기 밝기", "예상보다 어둡", "예상보다 멀다", "가속", "곡률", "오차", "모형가정"]
    },
    "neg": {}
  },
  "Q6": {
    "intent": [
        "fudge", "임의", "속임수", "꾸며낸", "반박", "평가", "대안", "한계",
        # ✅ 튜닝: '여러 관측/근거' 류 언급도 의도 인식에 포함
        "관측 결과", "실제 관측", "여러 관측", "다양한 관측", "여러 근거", "여러 증거"
    ],
    "pos": {
      "다중근거": ["회전곡선", "렌즈", "CMB", "BAO", "SN Ia", "은하단", "총알"],
      "대안 한계": ["MOND", "한계", "은하단", "총알"]
    },
    "neg": {
      "음모/단정": [
          "꾸며낸", "속임수", "사기", "증거 없다",
          "일부러 만든 개념", "일부로 만든 개념", "억지로 만든 개념",
          "관측 결과를 맞추기 위해", "관측 결과에 맞추려고",
          "틀린 이론을 유지하려고", "이론이 틀렸는데 유지", "틀린 이론을 감추기 위해"
      ]
    }
  },
  "Q7": {
    "intent": ["비유", "예시", "예를 들어", "처럼", "마치"],
    "pos": {
      "비유 제시": ["비유", "예시", "예를 들어", "처럼", "마치", "비슷", "풍선", "그물", "빵 반죽", "공기", "와인잔", "고무막", "수조", "상자"]
    },
    "neg": {}
  }
}

EM_GUARD = ["전자기파", "빛", "발광", "비발광", "전자기 상호작용", "직접"]
INDIRECT_GUARD = ["중력", "간접", "렌즈", "회전곡선", "질량", "증거", "질량지도"]

def near_guard(s, neg_terms, guards, window=36):
    """
    neg_terms 주변 window 글자 안에 guards(정개념 키워드)가 있으면
    해당 오개념 라벨은 무시(예: '직접 관측은 불가능하지만 중력으로 간접 검출').
    """
    for nt in neg_terms:
        for m in re.finditer(re.escape(nt.lower()), s):
            start, end = m.start(), m.end()
            left = max(0, start - window)
            right = min(len(s), end + window)
            if has_any(s[left:right], guards):
                return True
    return False

_SENT_SPLIT_RX = re.compile(r'(?<=[\.!?…])\s+|(?<=다[)\]"])\s+|\n+')

def split_sentences_ko(text: str):
    if not text:
        return []
    s = unicodedata.normalize("NFKC", str(text)).strip()
    if _KSS_OK:
        try:
            return [t.strip() for t in kss.split_sentences(s) if t.strip()]
        except Exception:
            pass
    return [t.strip() for t in re.split(_SENT_SPLIT_RX, s) if t.strip()]

# ★ 무응답 패턴 (예시 안 떠오름까지 포함)
NO_ANSWER_RX = re.compile(
    r"(몰라|모르겠|모릅니다|모름|기억\s*안\s*나|기억이\s*나지\s*않|잘\s*모르|생각이\s*안\s*나|확실치\s*않"
    r"|어렵습니다|어려워요|힘들어요|힘듭니다|이건\s*힘들"
    r"|적절한\s*예시가\s*떠오르지|예시가\s*떠오르지"
    r"|예시가\s*없어요|예시를\s*못\s*쓰겠|뭐라\s*써야\s*할지\s*모르겠)",
    re.I
)
EMOJI_JUNK_RX = re.compile(r"[^\w\s\uAC00-\uD7A3]+", re.UNICODE)

MEANINGFUL_KEYWORDS = [
    "암흑", "dark", "초신성", "sn ia", "가속", "허블", "렌즈", "회전", "회전 속도", "cmb", "우주 상수", "λ", "람다",
    "후보", "입자", "wimp", "윔프", "axion", "액시온", "macho", "마초",
]

def is_no_answer(text: str, qid: str | None = None) -> bool:
    s_raw = unicodedata.normalize("NFKC", str(text or "")).strip()
    s = EMOJI_JUNK_RX.sub(" ", s_raw).lower()
    if len(s) < 2:
        return True
    if not re.search(r"[가-힣a-zA-Z]", s):
        return True
    # 핵심 키워드가 하나라도 있으면 무응답으로 보지 않는다.
    if any(k in s for k in MEANINGFUL_KEYWORDS):
        return False
    return bool(NO_ANSWER_RX.search(s))

DE_UNIFORM_RX = re.compile(r"(균일|균등|일정한\s*밀도|우주\s*공간.*균|우주\s*상수|Λ)", re.I)

def rule_scan_by_q(answer: str, qid: str):
    s = _norm(answer)
    pos_hits, neg_hits = set(), {}
    has_no_ans_phrase = bool(NO_ANSWER_RX.search(s))
    q = Q_INTENT.get(qid, {})

    for cname, terms in q.get("pos", {}).items():
        if has_any(s, terms):
            pos_hits.add(cname)

    for label, terms in q.get("neg", {}).items():
        if not terms:
            continue
        found = [t for t in terms if t and t.lower() in s]
        if found:
            # 암흑/관측 관련 라벨은 주변에 정개념 키워드 있으면 가급적 패스
            if "암흑" in label or "관측" in label:
                if near_guard(s, found, EM_GUARD + INDIRECT_GUARD, window=36):
                    continue
            neg_hits.setdefault(label, []).extend(found)

    if qid == "Q1":
        # DM/DE 역할이 뒤집힌 경우에만 DM-DE 혼동으로 강하게 잡기
        if re.search(r"암흑\s*물질[^\.]{0,20}(가속|척력)", s) or \
           re.search(r"암흑\s*에너지[^\.]{0,20}(결속|인력|구조\s*형성)", s):
            neg_hits.setdefault("DM-DE 혼동", []).append("역할 전도")

    mislabels = list(neg_hits.keys()) or ["없음(정상)"]

    # 정개념 신호가 있으면 해당 오개념 라벨은 제거
    if ("DM=전자기 비상호작용(비발광)" in pos_hits) or ("dark=EM 비상호작용" in pos_hits):
        mislabels = [m for m in mislabels if "용어-암흑" not in m]
    if ("간접검출" in pos_hits) or ("DM=중력효과 간접 검출" in pos_hits):
        mislabels = [m for m in mislabels if m != "관측/검출 혼동"]
    if qid == "Q1" and DE_UNIFORM_RX.search(answer):
        mislabels = [m for m in mislabels if m != "분포/역할 오해"]

    # Q4에서 '후보 모름' 류는 천체 혼동 대신 무응답으로
    if qid == "Q4" and has_no_ans_phrase and "비후보/천체 혼동" in mislabels:
        if "무응답/비관련" not in mislabels:
            mislabels.append("무응답/비관련")

    return mislabels[:2], sorted(list(pos_hits)), neg_hits

# -------- 기타 스코어/증거 도우미 --------
RUBRIC_GUIDE = (
    "- score_judge(0–2): 질문 의도에 맞는 판단·근거 제시 여부\n"
    "- score_mech(0–5): DM/DE 정의·증거·분포·기전의 정확성/깊이(개념 대비·DE 신호·사슬 완결성 포함)\n"
    "- score_extend(0–3): 비유/응용/연결(관측 사례 등) 제시·정확도(비유 한계 언급 시 가산)\n"
    "감점 규칙) 명백한 오개념 포함 시 mech ≤ 3, judge ≤ 1. 무응답은 전부 0."
)

PROMPT_TEMPLATE = """너는 과학 교육 평가자다. 외부 지식 사용 금지.
아래 [근거]와 [응답]만 사용해 평가하고, 반드시 <<JSON>>…<</JSON>> 사이에 **유효한 JSON만** 출력하라.
설명/해설/코드는 절대 쓰지 말고 JSON 키만 포함하라.

[라벨 선택 규칙]
- 라벨은 다음 목록에서만 허용: [{labels}]
- 어느 라벨도 명확하지 않으면 ["없음(정상)"] **한 개만** 반환.
- 라벨은 최대 2개까지.
- "용어-암흑 의미혼동" 라벨은 dark/암흑이라는 이름의 **뜻**을
  "어두워서/검게 보여서/무서워서/눈에 안 보여서/정체불명이라서" 등으로 설명하는 경우에만 사용하라.
- 단순히 "미지의 힘", "정체를 모르는 존재"라고 표현하면서도
  암흑물질·암흑에너지의 물리적 역할과 증거를 비교적 정확히 설명한 경우에는
  "용어-암흑 의미혼동"으로 보지 말고 ["없음(정상)"]으로 처리하라.
- Q1에서 '아직 미지의 물질/에너지'라는 표현은, DM/DE의 역할·증거를 제대로 설명했다면
  **별도 오개념 라벨 없이** 정상으로 취급하라.
- 현재 문항 ID가 "Q7"인 경우, 오개념 라벨은 사용하지 말고
  ["무응답/비관련", "없음(정상)"] 중에서만 선택하라.
  비유의 적절성은 점수(score_extend)와 feedback에서만 평가하라.

[정개념 신호(탐지 시 긍정 플래그로 간주)]
- DM: EM 상호작용 미약/비발광, 중력효과 간접 검출(회전곡선·렌즈·질량지도·은하단 등)
- DE: 가속 팽창, 우주상수(Λ), 음의 압력, (거대 스케일) 균일 분포
- Q2: 회전곡선·렌즈·은하단·CMB/BAO 등 **다중 관측 근거**
- Q5: SN Ia 표준촛불 사슬(어둡게 관측→더 멀다→과거 감속→지금 가속)
- Q7: 비유 제시 + 비유의 **한계/적용 범위** 문장

[점수 규칙(요약)]
{rubric}
- 오개념 라벨(없음 제외)이 1개 이상이면 mech ≤ 3, judge ≤ 1로 상한.
- "무응답/비관련"이면 모든 점수 0.
- Q7: 암흑물질/암흑에너지 또는 우주 팽창과 연결된 비유가 하나라도 제시되면
  score_extend는 2 이상을 주고, 비유의 한계/적용 범위를 함께 설명한 경우에는
  score_extend를 3에 가깝게 준다.

[피드백 작성 규칙]
- 한국어 1–2문장, 학생이 바로 고칠 수 있게 **지시형**으로 작성.
- 축/오차/모형가정/비유의 한계 같은 **구체 단어**를 넣어 한 문장 내에 제시.
- Q7에서 비유는 있지만 한계 문장이 없으면,
  "이 비유가 어디까지 맞고 어디서 실제와 달라지는지 한 문장으로 한계를 써 보라"는
  권고 문장을 포함하라.
- Q1 응답에서 '미지/정체불명'이라는 말이 있어도, DM/DE의 정의·역할·증거를
  물리적으로 잘 대비했다면, 점수는 내용 위주로 주고 별도 오개념 라벨을 붙이지 말라.

[evidence_used 작성 규칙]
- [응답]에서만 1–2개의 **짧은 핵심 구절**을 직접 발췌(문장 또는 절).
- 참고문헌/저자/연도/파일키 등은 금지. 응답과 무관한 문구는 금지.

<<JSON>>
{{
 "mislabels": [],
 "scores": {{"judge":0,"mech":0,"extend":0}},
 "feedback": "",
 "evidence_used": []
}}
<</JSON>>

[현재 문항 ID]
{qid}

[근거]
{ctx}

[응답]
{ans}
"""

EVI_KEYWORDS_RX = re.compile(
    r"(회전곡선|평탄\s*회전|은하\s*헤일로|질량[-–]?\s*광도\s*비|중력렌즈|약\s*렌즈|질량\s*지도|"
    r"Ia\s*초신성|Ia형\s*초신성|Type\s*Ia|표준\s*촛불|"
    r"가속\s*팽창|CMB|우주\s*배경\s*복사|BAO|CDM|ΛCDM|람다\s*CDM|우주\s*상수|Λ)",
    re.I
)

ANALOGY_RX = re.compile(
    r"(비유|예시|예를\s*들어|처럼|마치|비슷|풍선|그물|빵\s*반죽|공기|와인잔|고무막|수조|상자)",
    re.I
)

_BAD_EVI_RX = re.compile(
    r'^(ref_id|title|authors|year|venue|q_links|quotes)\b|^"?(Rubin|Perlmutter|Riess)\d{4}|^[A-Z0-9_]{3,}$',
    re.I
)

def safe_parse_json(text: str):
    m = re.search(r"<<JSON>>(.*?)<</JSON>>", text, flags=re.S)
    block = m.group(1).strip() if m else text.strip()
    s = block.find("{"); e = block.rfind("}")
    if s != -1 and e != -1 and e > s:
        block = block[s:e+1]
    try:
        data = json.loads(block)
        sc = data.get("scores", {})
        data["scores"] = {k: int(sc.get(k, 0)) for k in ("judge", "mech", "extend")}
        data["mislabels"] = [x for x in (data.get("mislabels") or []) if x in ALLOWED_LABELS][:2]
        data["feedback"] = str(data.get("feedback", "")).strip()
        data["evidence_used"] = [str(x)[:160] for x in (data.get("evidence_used") or [])][:2]
        return data
    except Exception:
        return None

def split_sentences_and_rank(answer: str, max_spans=2):
    sents = split_sentences_ko(answer)
    ranked = []
    for i, s in enumerate(sents):
        hits = EVI_KEYWORDS_RX.findall(s)
        score = len(hits) - (0.5 if len(s) < 8 else 0.0) - (0.2 if "?" in s else 0.0)
        ranked.append((score, -len(s), i, s))
    ranked.sort(reverse=True)
    out, seen = [], set()
    for sc, _, _, s in ranked:
        if sc <= 0:
            break
        sig = re.sub(r"\W+", "", s.lower())
        if sig in seen:
            continue
        seen.add(sig); out.append(s[:160])
        if len(out) >= max_spans:
            break
    return out

def score_extend_heuristic(answer: str) -> int:
    a = re.sub(r"\s+", " ", str(answer or ""))
    has_analogy = bool(ANALOGY_RX.search(a))
    has_evi = bool(EVI_KEYWORDS_RX.search(a))
    return 2 if (has_analogy and has_evi) else (1 if has_analogy else 0)

def sanitize_evidence_used(answer, ev_list):
    ans_norm = re.sub(r'\s+', ' ', str(answer))
    out = []
    for s in (ev_list or []):
        s = str(s).strip().strip('"; ')
        if not s or _BAD_EVI_RX.search(s):
            continue
        if (s in ans_norm) or EVI_KEYWORDS_RX.search(s):
            if s not in out:
                out.append(s[:160])
    if len(out) < 2:
        for s in split_sentences_and_rank(answer, max_spans=2):
            if s not in out:
                out.append(s)
            if len(out) >= 2:
                break
    return out[:2]

def has_intent_terms(answer: str, qid: str) -> bool:
    q = Q_INTENT.get(qid, {})
    base = (q.get("intent") or []) + sum(q.get("pos", {}).values(), [])
    s = _norm(answer)
    return any(t.lower() in s for t in base[:120])

def qaware_score_boost(answer: str, qid: str, pos_hits, J, M, E):
    s = _norm(answer)

    if qid == "Q1":
        dm_ok = ("DM=전자기 비상호작용(비발광)" in pos_hits) or ("DM=중력효과 간접 검출" in pos_hits)
        de_ok = ("DE=가속팽창/음의 압력" in pos_hits)
        contrast = ("DM↔DE 대비" in pos_hits) or re.search(r"(대비|반면|vs|차이|구분)", s)

        if dm_ok and de_ok:
            J = max(J, 2); M = max(M, 4)
            if contrast:
                M = max(M, 5)
        if DE_UNIFORM_RX.search(answer):
            M = max(M, 4)

        attr_dm = bool(re.search(r"암흑\s*물질[^\.]{0,40}(인력|당기|끌어|잡아|중력)", s))
        rep_de  = bool(re.search(r"암흑\s*에너지[^\.]{0,40}(척력|밀어|밀리|가속|속도\s*를\s*높|더\s*빠르)", s))
        speed_contrast = bool(re.search(r"(팽창\s*속도|팽창\s*속도를)\s*(줄이|느리게|감속)|팽창\s*속도.*(높이|빠르게)", s))

        if attr_dm and rep_de:
            J = max(J, 1)
            M = max(M, 2)
        if attr_dm and rep_de and speed_contrast:
            J = max(J, 2)
            M = max(M, 3)

    elif qid == "Q2":
        evi_count = sum(1 for k in ("회전곡선 평탄", "렌즈/질량지도", "은하단 동역학", "CMB/구조형성") if k in pos_hits)
        if evi_count >= 2:
            J = max(J, 2); M = max(M, 4)
        elif evi_count == 1:
            J = max(J, 1); M = max(M, 3)

    # ✅ 튜닝: Q3에서 EM 비상호작용 정의가 잡힌 경우 최소 점수 보장
    elif qid == "Q3":
        if "dark=EM 비상호작용" in pos_hits:
            J = max(J, 1)
            M = max(M, 2)

    elif qid == "Q5":
        if "핵심 사슬" in pos_hits:
            J = max(J, 2); M = max(M, 4)

    # ✅ 튜닝: Q6에서 '다중근거' 또는 '여러 관측/근거/증거' 언급 시 최소 점수 보장
    elif qid == "Q6":
        multi = ("다중근거" in pos_hits) or re.search(r"(여러|다양한)\s*(관측|근거|증거)", s)
        if multi:
            J = max(J, 1)
            M = max(M, 2)

    elif qid == "Q7":
        if ("비유 제시" in pos_hits) and re.search(r"(한계|범위|어디까지|오해|주의|적용\s*범위)", s):
            E = max(E, 2)

    return J, M, E


For your information, Kss also supports mecab backend.
We recommend you to install mecab or konlpy.tag.Mecab for faster execution of Kss.
Please refer to following web sites for details:
- mecab: https://github.com/hyunwoongko/python-mecab-kor
- konlpy.tag.Mecab: https://konlpy.org/en/latest/api/konlpy.tag/#mecab-class



In [ ]:
# ============================================
# === Cell 9. 단일 답안 분석 함수         ===
# ============================================
def analyze_one(answer: str, qid: str, k_ctx=3, max_prompt_chars=3200):
    # 0) 완전 무응답이면 즉시 반환
    if is_no_answer(answer, qid=qid):
        return {
            "mislabels": ["무응답/비관련"],
            "scores": {"judge": 0, "mech": 0, "extend": 0},
            "feedback": "응답이 부족합니다. 질문에 맞춰 최소 1–2문장을 작성해 주세요.",
            "evidence_used": [],
            "concept_flags": []
        }

    # 1) 규칙 기반 1차 분석
    base_labels, concept_flags, neg_hits = rule_scan_by_q(answer, qid)

    # 2) LLM 호출 (REF 있으면 top-k 컨텍스트)
    ctx = topk_context(answer, k=k_ctx)
    prompt = PROMPT_TEMPLATE.format(
        labels=", ".join(ALLOWED_LABELS),
        rubric=RUBRIC_GUIDE,
        ctx=(ctx or "")[:1200],
        ans=str(answer)[:1500],
        qid=qid
    )[:max_prompt_chars]
    raw = llm_generate(prompt, max_new_tokens=260)
    data = safe_parse_json(raw) or {
        "mislabels": [],
        "scores": {"judge": 0, "mech": 0, "extend": 0},
        "feedback": "",
        "evidence_used": []
    }

    # 3) 라벨 병합 (규칙 우선 + HEAVY 라벨 가드 + Q별 필터)
    llm_raw = [x for x in (data.get("mislabels") or [])
               if x in Q_ALLOWED_MISLABELS.get(qid, ALLOWED_LABELS)]
    llm_filtered = []
    for lb in llm_raw:
        if lb in HEAVY_LABELS and lb not in neg_hits:
            # 규칙에서 안 잡힌 HEAVY 라벨은 과감히 버림(오탐 줄이기)
            continue
        llm_filtered.append(lb)

    ml = base_labels + llm_filtered
    ml = list(dict.fromkeys(ml))[:2]

    # 오개념이 하나라도 있으면 '없음(정상)'은 제거
    if any(x != "없음(정상)" for x in ml):
        ml = [x for x in ml if x != "없음(정상)"]
    if not ml:
        ml = ["없음(정상)"]

    # 4) 점수: LLM + 의도/핵심 키워드 하한 + Q별 부스팅
    J = data["scores"].get("judge", 0)
    M = data["scores"].get("mech", 0)
    E = data["scores"].get("extend", 0)

    if has_intent_terms(answer, qid) or any(k in concept_flags for k in POS_CORE.get(qid, set())):
        # 질문 의도 키워드·핵심 개념이 있으면 최소한의 하한 보장
        J = max(J, 1)
        M = max(M, 2)

    J, M, E = qaware_score_boost(answer, qid, set(concept_flags), J, M, E)

    # 5) 오개념 있을 때 상한
    if any(m != "없음(정상)" for m in ml):
        M = min(M, 3)
        J = min(J, 1)

    # 6) 확장점수 (비유/연결) — 휴리스틱과 최대값 사용
    E = max(E, score_extend_heuristic(answer))

    # 7) 피드백/증거
    fb = compose_feedback_qaware(qid, mislabels=ml, pos_hits=concept_flags)
    ev = sanitize_evidence_used(answer, data.get("evidence_used"))

    # 8) 무응답만 단독일 때 점수 0 고정
    if ml == ["무응답/비관련"]:
        J = M = E = 0

    # 9) Q5: SN Ia 핵심 키워드 없으면 기전/정의 불충분 보정
    s_raw = (answer or "")
    if qid == "Q5" and not re.search(r'(Ia|초신성|표준\s*촛불|겉보기\s*밝기|밝기|거리\s*지수|허블)', s_raw, flags=re.I):
        if "기전/정의 불충분" not in ml and "없음(정상)" in ml:
            ml = ["기전/정의 불충분"]

    return {
        "mislabels": ml,
        "scores": {"judge": int(J), "mech": int(M), "extend": int(E)},
        "feedback": fb,
        "evidence_used": ev,
        "concept_flags": concept_flags
    }


In [ ]:
# ============================================
# [Cell 10] 메인 루프 & CSV 저장
# ============================================
import time, gc, os, unicodedata
import pandas as pd
from tqdm.auto import tqdm
from IPython.display import display
import re

assert "df" in globals(), "[Cell 5] 이후 실행 필요: df가 없습니다."

FIXED_GRADE_COL = "현재 몇 학년에 재학 중이신가요?"
NAME_COL  = next((c for c in df.columns if "이름"   in str(c)), None)
GRADE_COL = FIXED_GRADE_COL if FIXED_GRADE_COL in df.columns else next(
    (c for c in df.columns if "학년" in str(c)), None
)
PHASE_COL  = "설문시점"   if "설문시점"   in df.columns else None
SOURCE_COL = "source_file" if "source_file" in df.columns else None  # ★ 원본 파일 이름

def clean_token(s):
    if pd.isna(s):
        return ""
    s = re.sub(r"`", "", str(s))
    return re.sub(r"\s+", " ", s).strip()

EXCLUDE_COLS = set(filter(None, [NAME_COL, GRADE_COL, PHASE_COL, SOURCE_COL]))
EXCLUDE_KEYWORDS = (
    "이름",
    "학년",
    "학번",
    "ID",
    "Email",
    "이메일",
    "제출",
    "타임스탬프",
    "시간",
    "점수",
    "설문",      # 설문시점 등
    "source",    # source_file 등
)

# 질문 컬럼 자동 탐지
q_cols_regex = [c for c in df.columns if re.match(r"^\s*\d+\.\s", str(c))]
if not q_cols_regex:
    cand = [
        c
        for c in df.columns
        if (
            df[c].dtype == "object"
            and c not in EXCLUDE_COLS
            and not any(k in str(c) for k in EXCLUDE_KEYWORDS)
        )
    ]
    Q_COLS = [c for c in cand if (df[c].astype(str).str.len().median() >= 5)]
else:
    Q_COLS = [c for c in q_cols_regex if c not in EXCLUDE_COLS]

if not Q_COLS:
    raise RuntimeError("분석대상 질문 컬럼을 찾지 못했습니다. 질문 컬럼명을 1–2개 알려주세요.")

print("분석대상 컬럼:", Q_COLS)

ID_CAND = ["학번", "ID", "이메일", "Email", "학생ID"]
id_col = next((c for c in ID_CAND if c in df.columns), None)

# 결과 저장 폴더: BASE_DIR/output 안에 저장
OUT_DIR = os.path.join(BASE_DIR, "output")
os.makedirs(OUT_DIR, exist_ok=True)

BATCH_SAVE_EVERY = 10
LIVE_SAVE_PATH   = os.path.join(OUT_DIR, "Survey_analysis_result_live.csv")
K_CTX            = 3
DRY_RUN_N        = None  # 부분 테스트시 숫자 넣기 (예: 20)

def _lead_number(header: str) -> int | None:
    s = unicodedata.normalize("NFKC", str(header or "")).strip()
    m = re.match(r"^\s*(\d+)\s*[\.)]", s)
    return int(m.group(1)) if m else None

Q_CANON = ["Q1", "Q2", "Q3", "Q4", "Q5", "Q6", "Q7"]

QMAP_REGEX = {
    "Q1": re.compile(r"(?=.*(암흑\s*물질|암흑\s*에너지))(?=.*(정의|차이|구분|구별|비교|대비))"),
    "Q2": re.compile(r"(?=.*(암흑\s*물질))(?=.*(증거|관측|회전곡선|렌즈|질량\s*대\s*광도|CMB))"),
    "Q3": re.compile(r"(?=.*(암흑|dark))(?=.*(이유|의미|왜|부르))"),
    "Q4": re.compile(r"(?=.*(후보|입자|WIMP|윔프|AXION|액시온|MACHO))", re.I),
    "Q5": re.compile(r"(?=.*(Ia|초신성|표준\s*촛불|거리\s*지수|허블))(?=.*(가속|어둡|곡률|오차|모형))", re.I),
    "Q6": re.compile(r"(?=.*(fudge|임의|속임수|꾸며낸))|(?=.*(타당|반박|대안|한계))", re.I),
    "Q7": re.compile(r"(?=.*(비유|예시|예를\s*들어))"),
}

def qid_from_header(header: str) -> tuple[str, int]:
    n = _lead_number(header)
    if n and 1 <= n <= 7:
        return Q_CANON[n - 1], n
    h = unicodedata.normalize("NFKC", str(header or "")).lower()
    for qid, rx in QMAP_REGEX.items():
        if rx.search(h):
            return qid, 0
    return "Q1", (n or 0)

rows = []
itr = list(df.iterrows())[:DRY_RUN_N] if DRY_RUN_N else df.iterrows()
total = (len(df) if not DRY_RUN_N else DRY_RUN_N) * len(Q_COLS)
pbar = tqdm(total=total, desc="분석 진행", unit="답안")

processed, t0 = 0, time.time()
for idx, row in itr:
    stu_name     = clean_token(row.get(NAME_COL, ""))    if NAME_COL   else ""
    stu_grade    = clean_token(row.get(GRADE_COL, ""))   if GRADE_COL  else ""
    survey_phase = clean_token(row.get(PHASE_COL, ""))   if PHASE_COL  else ""
    source_file  = clean_token(row.get(SOURCE_COL, ""))  if SOURCE_COL else ""
    aux_id       = clean_token(row.get(id_col, ""))      if id_col     else str(idx)

    for q in Q_COLS:
        ans = str(row.get(q, ""))
        qid, qno = qid_from_header(str(q))

        try:
            data = analyze_one(ans, qid=qid, k_ctx=K_CTX)
        except Exception as e:
            data = {
                "mislabels": ["무응답/비관련"],
                "scores": {"judge": 0, "mech": 0, "extend": 0},
                "feedback": f"분석 중 오류: {type(e).__name__}",
                "evidence_used": [],
                "concept_flags": [],
            }

        concept_tags = format_concept_tags(
            data.get("mislabels", []),
            data.get("concept_flags", []),
        )

        rows.append(
            {
                "source_file":   source_file,   # ★ 원본 설문 파일명
                "survey_phase":  survey_phase,  # 사전/사후/단일
                "student_id":    aux_id,
                "student_name":  stu_name,
                "student_grade": stu_grade,
                "question_id":   qid,           # Q1~Q7
                "question":      str(q),        # 원 질문 문장
                "answer":        ans,           # 학생 답안 원문
                "mislabels":     ", ".join(data.get("mislabels", [])),
                "score_judge":   int(data["scores"]["judge"]),
                "score_mech":    int(data["scores"]["mech"]),
                "score_extend":  int(data["scores"]["extend"]),
                "score_total":   int(
                    data["scores"]["judge"]
                    + data["scores"]["mech"]
                    + data["scores"]["extend"]
                ),
                "feedback":      data.get("feedback", ""),
                "evidence_used": "; ".join(data.get("evidence_used", [])),
                "concept_flags": concept_tags,
            }
        )

        processed += 1
        pbar.update(1)

        # 중간 저장(전체 live 파일은 하나로 유지)
        if processed % BATCH_SAVE_EVERY == 0:
            try:
                pd.DataFrame(rows).to_csv(
                    LIVE_SAVE_PATH, index=False, encoding="utf-8-sig"
                )
            except Exception as se:
                print(f"[경고] 라이브 저장 실패: {se}")

        # GPU/메모리 정리
        try:
            import torch
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
        except Exception:
            pass
        gc.collect()

pbar.close()
elapsed = time.time() - t0
avg = elapsed / max(1, processed)
print(f"총 {processed} 답안 처리 / {elapsed:.1f}s  (평균 {avg:.2f}s/답안)")

# 출력 컬럼 정리
out_cols = [
    "source_file",   # 원본 설문 파일명
    "survey_phase",  # 사전/사후/단일
    "student_id",
    "student_name",
    "student_grade",
    "question_id",
    "question",
    "answer",
    "mislabels",
    "score_judge",
    "score_mech",
    "score_extend",
    "score_total",
    "feedback",
    "evidence_used",
    "concept_flags",
]
out_df = pd.DataFrame(rows, columns=out_cols)

display(out_df.head(7))

# ===== 결과 저장: 파일 이름별로 분리 저장 =====
if "source_file" in out_df.columns and out_df["source_file"].notna().any():
    for sf, sub in out_df.groupby("source_file"):
        base = os.path.splitext(str(sf))[0] if sf else "Survey_analysis_result"
        out_path = os.path.join(OUT_DIR, f"{base}_analysis.csv")
        try:
            sub.to_csv(out_path, index=False, encoding="utf-8-sig")
            print("저장됨:", out_path)
        except Exception as e:
            print(f"[경고] 저장 실패({out_path}):", e)
else:
    # source_file 정보가 없는 경우, 예전처럼 하나만 저장
    fallback_path = os.path.join(OUT_DIR, "Survey_analysis_result.csv")
    try:
        out_df.to_csv(fallback_path, index=False, encoding="utf-8-sig")
        print("저장됨:", fallback_path)
    except Exception as e:
        print(f"[경고] 저장 실패({fallback_path}):", e)


분석대상 컬럼: ['1. 암흑물질(Dark Matter)과 암흑에너지(Dark Energy)를 각각 정의하고, 우주 전체에 미치는 영향의 가장 큰 차이점을 설명하세요.', '2. 암흑물질의 존재를 강력하게 지지하는 천문학적 관측 증거를 아는 대로 서술하세요.', "3. 암흑물질과 암흑에너지를 '암흑(Dark)'이라고 부르는 이유가 무엇이라고 생각하나요?", '4. 암흑물질의 후보로 언급되는 입자들의 특징을 설명하고, 왜 그런 특징을 보이는지 설명하세요.', "5. 1990년대 후반, 천문학자들이 우주의 '가속 팽창'을 발견하게 된 결정적인 관측은 무엇이었으며, 그 관측 결과가 왜 가속 팽창을 의미하는지 설명하세요.", "6. '암흑물질과 암흑에너지는 과학자들이 자신들의 이론이 틀린 것을 감추기 위해 임의로 도입한 개념(fudge factor)에 불과하다'라는 주장에 대해 어떻게 생각하시나요? 자신의 견해를 과학적 증거에 기반하여 논하세요.", '7. 학생들에게 암흑물질과 암흑에너지를 설명하기 위한 효과적인 비유나 예시를 한 가지 제시해 주세요.']


분석 진행:   0%|          | 0/147 [00:00<?, ?답안/s]

/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:623: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
/usr/local/lib/python3.12/dist-packages/pecab/_tokenizer.py:265: RuntimeWarning: overflow encountered in scalar add
  from_pos_data.costs[idx]
/usr/local/lib/python3.12/dist-packages/pecab/_tokenizer.py:274: RuntimeWarning: overflow encountered in scalar add
 

총 147 답안 처리 / 1242.0s  (평균 8.45s/답안)


,source_file,survey_phase,student_id,student_name,student_grade,question_id,question,answer,mislabels,score_judge,score_mech,score_extend,score_total,feedback,evidence_used,concept_flags
0,2025 교육연구프로그램 사전 설문(응답)....,사전,0,김지혜,대학교 2학년,Q1,1. 암흑물질(Dark Matter)과 암흑에너지(Dark Energy)를 각각 정...,"암흑물질은 눈에 보이지 않지만, 질량을 가진 물질을 의미한다. 암흑에너지는 우주를 ...",없음(정상),1,2,0,3,암흑물질=비발광(EM 상호작용 미약)·구조를 모음 / 암흑에너지=우주 가속 성분(음...,암흑에너지는 우주를 가속팽창하게 하는 에너지로 이용된다.,(정) DM↔DE 대비
1,2025 교육연구프로그램 사전 설문(응답)....,사전,0,김지혜,대학교 2학년,Q2,2. 암흑물질의 존재를 강력하게 지지하는 천문학적 관측 증거를 아는 대로 서술하세요.,"은하의 회전속도, 미세중력탐사",없음(정상),1,3,0,4,회전곡선 평탄·중력렌즈·은하단 동역학·CMB 중 1–2개를 축/오차와 함께 제시하세요.,,(정) 회전곡선 평탄
2,2025 교육연구프로그램 사전 설문(응답)....,사전,0,김지혜,대학교 2학년,Q3,3. 암흑물질과 암흑에너지를 '암흑(Dark)'이라고 부르는 이유가 무엇이라고 생각...,눈에 보이지 않기 때문에,없음(정상),0,0,0,0,‘직접’은 어렵지만 중력 효과로 ‘간접 검출’ 가능함을 한 줄로 정리하세요.,,
3,2025 교육연구프로그램 사전 설문(응답)....,사전,0,김지혜,대학교 2학년,Q4,"4. 암흑물질의 후보로 언급되는 입자들의 특징을 설명하고, 왜 그런 특징을 보이는지...","윔프, 윔프는 질량이 무겁고, 다른 입자들과 상호작용이 적기 때문이다.",없음(정상),1,2,0,3,WIMP/액시온 등 ‘입자’ 후보 1–2개 + ‘약한 상호작용·질량 범위’가 관측/...,,(정) 후보 명시
4,2025 교육연구프로그램 사전 설문(응답)....,사전,0,김지혜,대학교 2학년,Q5,"5. 1990년대 후반, 천문학자들이 우주의 '가속 팽창'을 발견하게 된 결정적인 ...","Ia 초신성 관측, 이 초신성은 표준광원인데, 그렇기 때문에 절대등급을 알 고 있다...",없음(정상),2,4,0,6,SN Ia 표준촛불: 더 어둡게 관측→더 멀다→과거 감속→지금 가속. 축/오차/모형...,"이때 초신성의 적색편이가 측정되어 가속팽창을 의미하게 된다.; Ia 초신성 관측, ...",(정) SN Ia 핵심 사슬
5,2025 교육연구프로그램 사전 설문(응답)....,사전,0,김지혜,대학교 2학년,Q6,6. '암흑물질과 암흑에너지는 과학자들이 자신들의 이론이 틀린 것을 감추기 위해 임...,"아니다. 은하의 회전속도, 은하단의 중력렌즈현상을 비추어보았을 때, 현대우주론을 발...",분포/역할 오해,1,2,0,3,회전곡선·렌즈·CMB/BAO·SN Ia 등 ‘독립 관측의 합치’ + 대안(MOND ...,"은하의 회전속도, 은하단의 중력렌즈현상을 비추어보았을 때, 현대우주론을 발전시키는 ...",(정) 다중 관측 근거; (정) 대안이론 한계; (부) 분포/역할 오해
6,2025 교육연구프로그램 사전 설문(응답)....,사전,0,김지혜,대학교 2학년,Q7,7. 학생들에게 암흑물질과 암흑에너지를 설명하기 위한 효과적인 비유나 예시를 한 가...,"눈에 보이지 않지만 영향을 주는 존재,, 녹아서 없어지는 것들? 그치만 영향을 주는...",없음(정상),0,0,0,0,"비유 1개를 제시하고, 이 비유가 어디까지 맞고 어디서 실제 암흑물질/에너지와 달라...",,


저장됨: /content/drive/MyDrive/colab/output/2025 교육연구프로그램 사전 설문(응답)_analysis.csv
저장됨: /content/drive/MyDrive/colab/output/2025 교육연구프로그램 사후 설문(응답)_analysis.csv
